In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='wisconsin',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_3',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([251, 251]) torch.Size([251, 251])
Training...
Epoch: 0001 train_loss= 1.38740 time= 0.01127
Epoch: 0002 train_loss= 1.38537 time= 0.00758
Epoch: 0003 train_loss= 1.38479 time= 0.00602
Epoch: 0004 train_loss= 1.38279 time= 0.00714
Epoch: 0005 train_loss= 1.38069 time= 0.00585
Epoch: 0006 train_loss= 1.37333 time= 0.00601
Epoch: 0007 train_loss= 1.36761 time= 0.00682
Epoch: 0008 train_loss= 1.35414 time= 0.00106
Epoch: 0009 train_loss= 1.34050 time= 0.00676
Epoch: 0010 train_loss= 1.31634 time= 0.00863
Epoch: 0011 train_loss= 1.29108 time= 0.00657
Epoch: 0012 train_loss= 1.26638 time= 0.00947
Epoch: 0013 train_loss= 1.23793 time= 0.00199
Epoch: 0014 train_loss= 1.24409 time= 0.00784
Epoch: 0015 train_loss= 1.23561 time= 0.00767
Epoch: 0016 train_loss= 1.18169 time= 0.00501
Epoch: 0017 train_loss= 1.19963 time= 0.01147
Epoch: 0018 train_loss= 1.14988 time= 0.00301
Epoch: 0019 train_loss= 1.15062 time= 0.00559
Epoch: 0020 train_loss= 1.13310 time= 0.00782


Epoch: 0190 train_loss= 0.40106 time= 0.00809
Epoch: 0191 train_loss= 0.36330 time= 0.00674
Epoch: 0192 train_loss= 0.37663 time= 0.00635
Epoch: 0193 train_loss= 0.34161 time= 0.00564
Epoch: 0194 train_loss= 0.39775 time= 0.00706
Epoch: 0195 train_loss= 0.36812 time= 0.00568
Epoch: 0196 train_loss= 0.32495 time= 0.00692
Epoch: 0197 train_loss= 0.36136 time= 0.00701
Epoch: 0198 train_loss= 0.33665 time= 0.00607
Epoch: 0199 train_loss= 0.38237 time= 0.00698
Epoch: 0200 train_loss= 0.33575 time= 0.00700
Testing model...
Masking test edges...
torch.Size([251, 251]) torch.Size([251, 251])
Training...
Epoch: 0001 train_loss= 1.38659 time= 0.00702
Epoch: 0002 train_loss= 1.38570 time= 0.00575
Epoch: 0003 train_loss= 1.38373 time= 0.00608
Epoch: 0004 train_loss= 1.38160 time= 0.00492
Epoch: 0005 train_loss= 1.37690 time= 0.00600
Epoch: 0006 train_loss= 1.37054 time= 0.00701
Epoch: 0007 train_loss= 1.35794 time= 0.00499
Epoch: 0008 train_loss= 1.34446 time= 0.00700
Epoch: 0009 train_loss= 1.321

Epoch: 0175 train_loss= 0.45935 time= 0.00900
Epoch: 0176 train_loss= 0.38494 time= 0.00800
Epoch: 0177 train_loss= 0.45209 time= 0.01000
Epoch: 0178 train_loss= 0.44413 time= 0.00574
Epoch: 0179 train_loss= 0.40670 time= 0.00715
Epoch: 0180 train_loss= 0.37825 time= 0.00684
Epoch: 0181 train_loss= 0.42721 time= 0.00800
Epoch: 0182 train_loss= 0.49175 time= 0.00600
Epoch: 0183 train_loss= 0.44267 time= 0.00802
Epoch: 0184 train_loss= 0.41747 time= 0.00699
Epoch: 0185 train_loss= 0.44514 time= 0.00700
Epoch: 0186 train_loss= 0.41888 time= 0.00700
Epoch: 0187 train_loss= 0.44395 time= 0.00700
Epoch: 0188 train_loss= 0.45331 time= 0.00699
Epoch: 0189 train_loss= 0.40618 time= 0.00607
Epoch: 0190 train_loss= 0.41356 time= 0.00693
Epoch: 0191 train_loss= 0.42974 time= 0.00706
Epoch: 0192 train_loss= 0.40737 time= 0.00702
Epoch: 0193 train_loss= 0.40497 time= 0.00899
Epoch: 0194 train_loss= 0.40815 time= 0.00701
Epoch: 0195 train_loss= 0.38301 time= 0.00794
Epoch: 0196 train_loss= 0.38344 ti

Epoch: 0161 train_loss= 0.45578 time= 0.00736
Epoch: 0162 train_loss= 0.44950 time= 0.00800
Epoch: 0163 train_loss= 0.51903 time= 0.00761
Epoch: 0164 train_loss= 0.51991 time= 0.00701
Epoch: 0165 train_loss= 0.41778 time= 0.00500
Epoch: 0166 train_loss= 0.50709 time= 0.00501
Epoch: 0167 train_loss= 0.40596 time= 0.00700
Epoch: 0168 train_loss= 0.46034 time= 0.00699
Epoch: 0169 train_loss= 0.38110 time= 0.00702
Epoch: 0170 train_loss= 0.45600 time= 0.00473
Epoch: 0171 train_loss= 0.43200 time= 0.00634
Epoch: 0172 train_loss= 0.47121 time= 0.00586
Epoch: 0173 train_loss= 0.40401 time= 0.00798
Epoch: 0174 train_loss= 0.43389 time= 0.00606
Epoch: 0175 train_loss= 0.41106 time= 0.00700
Epoch: 0176 train_loss= 0.41988 time= 0.00587
Epoch: 0177 train_loss= 0.41073 time= 0.00279
Epoch: 0178 train_loss= 0.38258 time= 0.00677
Epoch: 0179 train_loss= 0.37745 time= 0.00745
Epoch: 0180 train_loss= 0.40565 time= 0.00718
Epoch: 0181 train_loss= 0.37582 time= 0.01082
Epoch: 0182 train_loss= 0.41729 ti

Epoch: 0161 train_loss= 0.44007 time= 0.00500
Epoch: 0162 train_loss= 0.36949 time= 0.00546
Epoch: 0163 train_loss= 0.45391 time= 0.00598
Epoch: 0164 train_loss= 0.38838 time= 0.00507
Epoch: 0165 train_loss= 0.42584 time= 0.00447
Epoch: 0166 train_loss= 0.53609 time= 0.00607
Epoch: 0167 train_loss= 0.45129 time= 0.00500
Epoch: 0168 train_loss= 0.40915 time= 0.00446
Epoch: 0169 train_loss= 0.56881 time= 0.00598
Epoch: 0170 train_loss= 0.42837 time= 0.00601
Epoch: 0171 train_loss= 0.53377 time= 0.00699
Epoch: 0172 train_loss= 0.51030 time= 0.00603
Epoch: 0173 train_loss= 0.44657 time= 0.00592
Epoch: 0174 train_loss= 0.42102 time= 0.00708
Epoch: 0175 train_loss= 0.48995 time= 0.00599
Epoch: 0176 train_loss= 0.52961 time= 0.00698
Epoch: 0177 train_loss= 0.43752 time= 0.00402
Epoch: 0178 train_loss= 0.44629 time= 0.00593
Epoch: 0179 train_loss= 0.41300 time= 0.00683
Epoch: 0180 train_loss= 0.42315 time= 0.00600
Epoch: 0181 train_loss= 0.40899 time= 0.00800
Epoch: 0182 train_loss= 0.43394 ti

...

Epoch: 0158 train_loss= 0.45790 time= 0.00675
Epoch: 0159 train_loss= 0.46232 time= 0.00200
Epoch: 0160 train_loss= 0.43573 time= 0.00100
Epoch: 0161 train_loss= 0.44077 time= 0.00729
Epoch: 0162 train_loss= 0.39435 time= 0.00584
Epoch: 0163 train_loss= 0.42336 time= 0.00454
Epoch: 0164 train_loss= 0.43069 time= 0.00400
Epoch: 0165 train_loss= 0.41700 time= 0.00391
Epoch: 0166 train_loss= 0.41954 time= 0.00491
Epoch: 0167 train_loss= 0.42064 time= 0.00455
Epoch: 0168 train_loss= 0.43988 time= 0.00400
Epoch: 0169 train_loss= 0.42984 time= 0.00400
Epoch: 0170 train_loss= 0.43607 time= 0.00500
Epoch: 0171 train_loss= 0.37362 time= 0.00100
Epoch: 0172 train_loss= 0.38559 time= 0.00200
Epoch: 0173 train_loss= 0.41274 time= 0.00521
Epoch: 0174 train_loss= 0.41863 time= 0.00331
Epoch: 0175 train_loss= 0.40777 time= 0.00503
Epoch: 0176 train_loss= 0.37348 time= 0.00880
Epoch: 0177 train_loss= 0.40582 time= 0.00203
Epoch: 0178 train_loss= 0.39295 time= 0.01078
Epoch: 0179 train_loss= 0.36673 ti

Epoch: 0143 train_loss= 0.47683 time= 0.00800
Epoch: 0144 train_loss= 0.44997 time= 0.01200
Epoch: 0145 train_loss= 0.47455 time= 0.00700
Epoch: 0146 train_loss= 0.43470 time= 0.00700
Epoch: 0147 train_loss= 0.46088 time= 0.00600
Epoch: 0148 train_loss= 0.48069 time= 0.00803
Epoch: 0149 train_loss= 0.44024 time= 0.00501
Epoch: 0150 train_loss= 0.45909 time= 0.00505
Epoch: 0151 train_loss= 0.45970 time= 0.00489
Epoch: 0152 train_loss= 0.42591 time= 0.00472
Epoch: 0153 train_loss= 0.44461 time= 0.00701
Epoch: 0154 train_loss= 0.42879 time= 0.00548
Epoch: 0155 train_loss= 0.40020 time= 0.00572
Epoch: 0156 train_loss= 0.46771 time= 0.00705
Epoch: 0157 train_loss= 0.42610 time= 0.00595
Epoch: 0158 train_loss= 0.45892 time= 0.00617
Epoch: 0159 train_loss= 0.41619 time= 0.00606
Epoch: 0160 train_loss= 0.44284 time= 0.00811
Epoch: 0161 train_loss= 0.43584 time= 0.00690
Epoch: 0162 train_loss= 0.46652 time= 0.00600
Epoch: 0163 train_loss= 0.37242 time= 0.00692
Epoch: 0164 train_loss= 0.45091 ti

Epoch: 0126 train_loss= 0.45582 time= 0.00700
Epoch: 0127 train_loss= 0.43774 time= 0.00805
Epoch: 0128 train_loss= 0.47106 time= 0.00800
Epoch: 0129 train_loss= 0.48377 time= 0.00696
Epoch: 0130 train_loss= 0.44670 time= 0.00700
Epoch: 0131 train_loss= 0.48068 time= 0.00607
Epoch: 0132 train_loss= 0.54723 time= 0.00623
Epoch: 0133 train_loss= 0.51918 time= 0.00893
Epoch: 0134 train_loss= 0.47226 time= 0.00700
Epoch: 0135 train_loss= 0.55591 time= 0.00900
Epoch: 0136 train_loss= 0.47972 time= 0.00700
Epoch: 0137 train_loss= 0.49762 time= 0.00800
Epoch: 0138 train_loss= 0.47267 time= 0.00700
Epoch: 0139 train_loss= 0.48256 time= 0.00800
Epoch: 0140 train_loss= 0.45645 time= 0.00700
Epoch: 0141 train_loss= 0.42553 time= 0.00700
Epoch: 0142 train_loss= 0.46046 time= 0.00730
Epoch: 0143 train_loss= 0.43325 time= 0.00470
Epoch: 0144 train_loss= 0.40530 time= 0.00778
Epoch: 0145 train_loss= 0.47089 time= 0.00700
Epoch: 0146 train_loss= 0.46693 time= 0.00700
Epoch: 0147 train_loss= 0.41008 ti

Epoch: 0114 train_loss= 0.48720 time= 0.00600
Epoch: 0115 train_loss= 0.55260 time= 0.00620
Epoch: 0116 train_loss= 0.53861 time= 0.00898
Epoch: 0117 train_loss= 0.48824 time= 0.00700
Epoch: 0118 train_loss= 0.59092 time= 0.00590
Epoch: 0119 train_loss= 0.44952 time= 0.00589
Epoch: 0120 train_loss= 0.48005 time= 0.00569
Epoch: 0121 train_loss= 0.45519 time= 0.00461
Epoch: 0122 train_loss= 0.42667 time= 0.00450
Epoch: 0123 train_loss= 0.46989 time= 0.00302
Epoch: 0124 train_loss= 0.46837 time= 0.00475
Epoch: 0125 train_loss= 0.45877 time= 0.01178
Epoch: 0126 train_loss= 0.42316 time= 0.00126
Epoch: 0127 train_loss= 0.45538 time= 0.00659
Epoch: 0128 train_loss= 0.42264 time= 0.00800
Epoch: 0129 train_loss= 0.43762 time= 0.00543
Epoch: 0130 train_loss= 0.43705 time= 0.01152
Epoch: 0131 train_loss= 0.42660 time= 0.00185
Epoch: 0132 train_loss= 0.45207 time= 0.00206
Epoch: 0133 train_loss= 0.41599 time= 0.01010
Epoch: 0134 train_loss= 0.40209 time= 0.00671
Epoch: 0135 train_loss= 0.43753 ti

Epoch: 0104 train_loss= 0.47479 time= 0.00592
Epoch: 0105 train_loss= 0.47339 time= 0.00600
Epoch: 0106 train_loss= 0.50336 time= 0.00400
Epoch: 0107 train_loss= 0.49206 time= 0.00504
Epoch: 0108 train_loss= 0.49761 time= 0.00741
Epoch: 0109 train_loss= 0.41513 time= 0.00661
Epoch: 0110 train_loss= 0.43676 time= 0.00700
Epoch: 0111 train_loss= 0.48692 time= 0.00594
Epoch: 0112 train_loss= 0.46375 time= 0.00800
Epoch: 0113 train_loss= 0.42756 time= 0.00567
Epoch: 0114 train_loss= 0.51836 time= 0.00700
Epoch: 0115 train_loss= 0.46482 time= 0.00701
Epoch: 0116 train_loss= 0.46424 time= 0.00699
Epoch: 0117 train_loss= 0.45184 time= 0.00800
Epoch: 0118 train_loss= 0.47347 time= 0.00700
Epoch: 0119 train_loss= 0.47167 time= 0.00668
Epoch: 0120 train_loss= 0.43174 time= 0.00600
Epoch: 0121 train_loss= 0.50068 time= 0.00822
Epoch: 0122 train_loss= 0.43981 time= 0.00677
Epoch: 0123 train_loss= 0.46366 time= 0.00669
Epoch: 0124 train_loss= 0.45457 time= 0.00709
Epoch: 0125 train_loss= 0.43919 ti